In [ ]:
pip install bs4 lxml

In [1]:
from lxml import html
import requests

In [3]:
#https://sudact.ru/arbitral/doc/?page=4&arbitral-judge=&arbitral-court=&arbitral-case_doc=&arbitral-date_from=01.09.2023&arbitral-lawchunkinfo=&arbitral-txt=&arbitral-date_to=30.09.2023&_=1700647317938&arbitral-region=


#https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from={start_date}&arbitral-date_to={end_date}&arbitral-region=&arbitral-court=&arbitral-judge=&_=1700650277067


In [2]:
from datetime import datetime, timedelta

pattern_link = "https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from={start_date}&arbitral-date_to={end_date}&arbitral-region=&arbitral-court=&arbitral-judge="


def get_links_sudact(link = pattern_link,delta_days = 30, finish_date = datetime(2023,1,1), start_date_pattern = "{start_date}", end_date_pattern = "{end_date}"):
    current_datetime = datetime.now()
    start_date = current_datetime - timedelta(days=delta_days)
    end_date = current_datetime

    links = []

    while start_date.date() > finish_date.date():
        end_date = start_date
        start_date = start_date - timedelta(days=1)

        result_link = link.replace(start_date_pattern, str(start_date.date()))
        result_link = result_link.replace(end_date_pattern, str(end_date.date()))

        links.append(result_link)

    return links


In [3]:

links = get_links_sudact()

In [4]:
len(links)

296

In [22]:
from bs4 import BeautifulSoup
from time import sleep


def get_documents_link_from_page(link):

    result = []

    json_content = get_json_content(link)

    if "content" not in json_content:
        sleep(5)
        get_documents_link_from_page(link)
    
    content = json_content["content"]

    bs = BeautifulSoup(content,"lxml")
    bs_results = bs.find('ul', 'results')


    if(bs_results is None):
        return result

    documents = bs_results.find_all('li')


    for document in documents:
        try:
            justice = document.find('div', 'b-justice')
            document_link = document.find('a')
            record = {
                'document_link': document_link.get('href'),
                'title': document_link.text,
                'justice': justice.text
            }
            result.append(record)
        except:
            print(f"Document error: {document}")

    return result


    


In [8]:
import random

def get_json_content(link):
    try:
        page = requests.get(link)
        return page.json()
    except:
        sleep(random.randrange(5,15))
        print(f"Link error: {link}")
        return get_json_content(link)

In [7]:
for link in links[:10]:
    print(link)

https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-23&arbitral-date_to=2023-10-24&arbitral-region=&arbitral-court=&arbitral-judge=
https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-22&arbitral-date_to=2023-10-23&arbitral-region=&arbitral-court=&arbitral-judge=
https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-21&arbitral-date_to=2023-10-22&arbitral-region=&arbitral-court=&arbitral-judge=
https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-20&arbitral-date_to=2023-10-21&arbitral-region=&arbitral-court=&arbitral-judge=
https://sudact.ru/arbitral/doc_ajax/?page={page_n}&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-19&arbitra

In [18]:
import re

def get_json_result_total_count(link):
    try:
        json_result = get_json_content(link)
        total_str = json_result["total_found"].replace(' ', '')
        document_count = 0
        numbers = re.findall(r"\d+",total_str)
        if(len(numbers) > 0):
            document_count = numbers[0]
        return json_result, document_count
    except:
        sleep(5)
        print(f"Error gettings total found: {json_result}")
        return get_json_result_total_count(link)

In [19]:
print(f"Links count: {len(links)}")

Links count: 296


In [23]:
documents_result = []
link_n = 0

In [24]:

for link in links:
    i = 1
    print
    page_number_pattern = "{page_n}"
    start_link = link.replace(page_number_pattern, str(i))
    
    json_result, document_count = get_json_result_total_count(start_link)

    if document_count == 0:
        continue

    page_count = int(int(document_count)/10) + 1
    start_doc = get_documents_link_from_page(start_link)
    documents_result.extend(start_doc)

    print(f"Link {start_link}")
    print(f"Doc count:  {document_count}")
    print(f"Page count:  {page_count}")
    

    if page_count == 1:
        print(f"Results count:  {len(documents_result)}")
        continue

    for x in range(page_count):
        i = i+1
        parse_link = link.replace(page_number_pattern, str(i))
        documents_result.extend(get_documents_link_from_page(parse_link))

    print(f"Results count:  {len(documents_result)}")
    print(f"------------------------------------------")
    link_n = link_n + 1





Link https://sudact.ru/arbitral/doc_ajax/?page=1&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-23&arbitral-date_to=2023-10-24&arbitral-region=&arbitral-court=&arbitral-judge=
Doc count:  3
Page count:  1
Results count:  3
Link https://sudact.ru/arbitral/doc_ajax/?page=1&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-22&arbitral-date_to=2023-10-23&arbitral-region=&arbitral-court=&arbitral-judge=
Doc count:  2
Page count:  1
Results count:  5
Link https://sudact.ru/arbitral/doc_ajax/?page=1&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-19&arbitral-date_to=2023-10-20&arbitral-region=&arbitral-court=&arbitral-judge=
Doc count:  2
Page count:  1
Results count:  7
Link https://sudact.ru/arbitral/doc_ajax/?page=1&arbitral-txt=&arbitral-case_doc=&arbitral-lawchunkinfo=&arbitral-date_from=2023-10-18&arbitral-date_to=2023-10-19&arbitral-region=&arbitral-court=&arbitral-judge=
Doc count: 

In [ ]:
len(documents_result)

In [ ]:
import json

def save_result(filename, json_result):
    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(json_result, json_file, ensure_ascii=False, indent=4)



In [ ]:
save_result('sudact_2023_documents.json', documents_result)
save_result('sudact_2023_links.json', links)